In [5]:
%load_ext autoreload
%autoreload 2

import os
import joblib
import youtube_crawler.utils as yc_utils
import keyword_generation.utils as kg_utils

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

# -----------------------------------
# Paths

query_path = os.path.join("data", "queries")

# -----------------------------------
# Flags & global settings

compute_new_keywords = True
NUMBER_OF_QUERIES_TO_GENERATE = 2


crawl_youtube = True

NUMBER_OF_PATHS_TO_COLLECT_PER_KEYWORD = 20
NUMBER_OF_SEARCH_RESULTS_TO_CONSIDER = 10
NUMBER_OF_RELATED_VIDEOS_TO_CONSIDER = 10
NUMBER_OF_RELATED_VIDEOS_TO_VISIT_DEPTH = 4  # number of recommendations that are collected


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Create or load personas

In [ ]:

"""
for creating a persona we can use Attitude Based Audience Targeting (Kefford et al. 2023; Blasi Casagran, Vermeulen 2021) which is used in PMT (Political Microtargeting) and includes:

demographic factors (Group identifiers like Age, Location, Income etc.)
interests (f.e. party membership, opinions on issues)
behavior (feelings, values)
connection (networks)
"""

# A persona is a list of attributes formulated in natural language
personas = {
    "example" : "(Attributes: likes to cook, is male, is a middle aged US citizen)"
}

### Generate keywords based on personas

In [9]:

model_name = "distilgpt2"

query_store_path = os.path.join(os.path.join(query_path, model_name))

if not os.path.exists(query_store_path):
    os.makedirs(query_store_path)

query_store_path = os.path.join(query_store_path, "persona_query_dict")

if compute_new_keywords:

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    persona_query_dict = kg_utils.generate_keywords(model, tokenizer, personas, NUMBER_OF_QUERIES_TO_GENERATE)

    joblib.dump(persona_query_dict, query_store_path)

    for persona in persona_query_dict:
        print(f"-- persona: {persona} --")

        for idx, query in enumerate(persona_query_dict[persona]):
            print(f"\nId {idx}")
            text = query.replace("\n", " ")
            print(text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


### Crawl Youtube

In [11]:
if crawl_youtube:
    persona_query_dict = joblib.load(query_store_path)

    for persona in persona_query_dict:

        keywords = persona_query_dict[persona]

        yc_utils.crawl_youtube(keywords,
                               NUMBER_OF_PATHS_TO_COLLECT_PER_KEYWORD, NUMBER_OF_SEARCH_RESULTS_TO_CONSIDER,
                               NUMBER_OF_RELATED_VIDEOS_TO_CONSIDER, NUMBER_OF_RELATED_VIDEOS_TO_VISIT_DEPTH,
                               "data", persona)


b'Impersonate a person that has the following attributes: (Attributes: likes to cook, is male, is a middle aged US citizen) and now create a queries you would search in youtube aligning with your personality. This is an important feature in' --- 0 --- b'https://youtube.de/results?search_query=Impersonate%20a%20person%20that%20has%20the%20following%20attributes:%20(Attributes:%20likes%20to%20cook,%20is%20male,%20is%20a%20middle%20aged%20US%20citizen)%20and%20now%20create%20a%20queries%20you%20would%20search%20in%20youtube%20aligning%20with%20your%20personality.%20This%20is%20an%20important%20feature%20in'


OSError: [Errno 22] Invalid argument: 'data\\crawled_pages\\example\\0attributes_%20(Attributes_%20likes%20to%20cook,%20is%20male,%20is%20a%20middle%20aged%20US%20citizen)%20and%20now%20create%20a%20queries%20you%20would%20search%20in%20youtube%20aligning%20with%20your%20personality_%20This%20is%20an%20important%20feature%20in.html'